# Projet IA

In [ ]:
import scipy
import numpy
import math
import sklearn
import pandas as pds
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
#import sergio_peignier as sp

## Attribute selection

In [ ]:
sparse_df = pds.read_csv("./datas/Data-IA-World-Development-Indicator.txt", sep="\t", header=0)

In [ ]:
sparse_df.shape

In [ ]:
sparse_df.head()

In [ ]:
sparse_df.tail()

In [ ]:
# Un peu de statistiques :
sparse_df.describe()

In [ ]:
#Fonction qui supprime une colonne (attribut) si le nb de NaN >= limit

def delete_NaN_col(df, limit):
    NaN_col = df.isna().sum()
    tmp = df.copy(deep=True) #temporary df
    
    for i in range(df.shape[1]):
        if NaN_col[i] >= limit:
            df = df.drop(columns = tmp.columns[i])
            
    return(df)

In [ ]:
sparse_df2 = delete_NaN_col(sparse_df, 40)

In [ ]:
# On enlèvre les colonnes 'Time' et 'Time Code' qui ne nous interessent pas ici
def remove_useless_col(df):
    df = df.drop(columns = ["Time", "Time Code", "Country Code"])
    return (df)

In [ ]:
sparse_df2 = remove_useless_col(sparse_df2)

In [ ]:
sparse_df2.isna().sum()

In [ ]:
sparse_df2.isna().sum(axis=1)

In [ ]:
#Fonction qui supprime une ligne (object) si le nb de NaN >= limit

def delete_NaN_row(df, limit):
    NaN_row = df.isna().sum(axis=1)
    tmp = df.copy(deep=True) #temporary df
    
    for i in range(df.shape[0]):
        if NaN_row[i] >= limit:
            df = df.drop([i], axis = 0)
            
    return(df)

In [ ]:
sparse_df3 = delete_NaN_row(sparse_df2, 1)

In [ ]:
sparse_df3

In [ ]:
#On remplace les indexes des ligne {0, 1, 2, ..n} par les noms de pays
#Puis on supprime la colonne "Countru Name" pour n'avoir plus que des valeurs numériques
sparse_df3.index = sparse_df3['Country Name']
sparse_df3 = sparse_df3.drop(columns = ['Country Name'])

In [ ]:
sparse_df3.head()

# Études des correlations :

In [ ]:
corr_df = sparse_df3.corr()
corr_df[:5]

In [ ]:
# Représentation graphique des correlations
plt.figure(figsize= (15, 12))
sns.heatmap(corr_df,annot=True)
plt.show()

In [ ]:
sns.clustermap(corr_df,
               figsize= (16, 12),
               annot=True,
               dendrogram_ratio=(0.1, 0.2),
               row_cluster=False)
plt.show()

In [ ]:
def delete_corr(df, limit):
    #On construit notre matrice de correlation en valeur absolue
    corr_matrix = df.corr().abs()
    
    #Triangle supérieur de la matrice de corrélation : 
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > limit)]
    df = df.drop(df[to_drop], axis=1)
    return(df)

In [ ]:
df = delete_corr(sparse_df3, 0.55)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize= (15, 12))
sns.heatmap(df.corr().abs(), annot = True)
plt.show()

In [ ]:
#On ne sait pas vraiment à quoi correspond cet attribut
#De plus il faudrait normaliser ses valeurs, sauf que nous ne connaissons aucun référentiel pour faire cela
df = df.drop(columns=["Net domestic credit (current LCU) [FM.AST.DOMS.CN]"]) 

In [ ]:
# On renomme les labels des colonnes pour plus de fluidité
df = df.rename(columns={"Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]" : "Clean fuels and technologies for cooking acces (% pop)"})
df = df.rename(columns={"Aquaculture production (metric tons) [ER.FSH.AQUA.MT]" : "Aquaculture prod (metric tons)"})
df = df.rename(columns={"Compulsory education, duration (years) [SE.COM.DURS]" : "Compulsory education duration (years)"})
df = df.rename(columns={"Death rate, crude (per 1,000 people) [SP.DYN.CDRT.IN]" : "Death rate (per 10e3 )"})
df = df.rename(columns={"Incidence of tuberculosis (per 10e5 people) [SH.TBS.INCD]" : "Incidence of tuberculosis (per 100 000)"})
df = df.rename(columns={"Number of infant deaths [SH.DTH.IMRT]" : "Number of infant deaths"})
df = df.rename(columns={"Preprimary education, duration (years) [SE.PRE.DURS]" : "Preprimary education duration (years)"})

In [ ]:
df.head()

# Normalisation relative :

In [ ]:
def remove_country(df, df_ref):
    tmp = df.copy(deep = True)
    for i, country in  enumerate( tmp["Country Name"]):
        if country not in df_ref.index.values:
            df = df.drop(tmp.index[i], axis = 0)
    return (df)

In [ ]:
population_size = pds.read_csv("./datas/Popula-schtroumpf/Population-size-per-country.txt", sep="\t", header=0)
population_size = remove_useless_col(population_size)
population_size = delete_NaN_row(population_size, 1)
population_size = remove_country(population_size, df)

In [ ]:
surfaces = pds.read_csv("./datas/Surfa-schtroumpf/Country-surfaces.txt", sep="\t", header=0)
surfaces = remove_useless_col(surfaces)
surfaces = delete_NaN_row(surfaces, 1)
surfaces = remove_country(surfaces, df)

In [ ]:
print (len(population_size) == len(df), len(surfaces)== len(df))

In [ ]:
new_df = df.copy(deep = True)
print("Avant :", "\n")
new_df.head()

In [ ]:
new_df.iloc[:, 1] =  new_df.iloc[:, 1].values/surfaces.iloc[:, 1].values
new_df.iloc[:, 5] =  new_df.iloc[:, 5].values/population_size.iloc[:, 1].values
print("Après :", "\n")
new_df.head()

# Normalisation par centrage - réduction :

\begin{equation*}
    X = \frac{X - \mu}{\sigma}
\end{equation*}

In [ ]:
sns.pairplot(data=new_df)
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
sns.boxplot(data = new_df)
plt.show()

#On voit que les ordres de grandeur de nos données ainsi que leur variances ne sont pas du tout homogènes.
#On va donc les normaliser (centrer - réduire) pour corriger ce défaut  

In [ ]:
# Fonction qui centre et réduit une colonne de données pour les normaliser
def normalizer (data):
    # Quelques statistiques :
    mean = np.mean(data)
    var = np.var(data)
    sd = math.sqrt(var)
    
    norm = []
    for x in data:
        norm.append((x-mean)/sd)
    
    return (norm)

In [ ]:
def norm_whole_df (df):
    normed_df = df.copy(deep = True)
    for i in df.columns:
        normed_df[i] = normalizer(df[i])
        
    return (normed_df)

In [ ]:
normed_df = norm_whole_df(new_df)

In [ ]:
normed_df.head()

In [ ]:
normed_df.shape

In [ ]:
plt.figure(figsize=(12,15))
sns.boxplot(data = normed_df)
plt.show()

In [ ]:
from scipy import stats

In [ ]:
def search_outliers_row(df, limit):
    z_scores = stats.zscore(normed_df) #calcule le z-score de df
    abs_z_scores = np.abs(z_scores) # on passe en valeurs absolues
    filtered_outliers = (abs_z_scores < limit).all(axis=1) # on ne garde que les lignes avec des valeurs > limit
    new_df = df[filtered_outliers] #nouveau df
    return (new_df)

In [ ]:
df2 = search_outliers_row(normed_df, 2.4)

In [ ]:
df2.shape

In [ ]:
plt.figure(figsize=(12,15))
sns.boxplot(data = df2)
plt.show()

In [ ]:
#Pairplot après standardisation
sns.pairplot(data=df2)
plt.show()

## Clustering :

### Dataset clustering using K-means

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [ ]:
# Création d'un object KMeans :
km = KMeans(n_clusters=4, init='k-means++',n_init=10, random_state=50, max_iter=300,).fit(df2)

In [ ]:
clusters = km.labels_

In [ ]:
# On stocke les centroides obtenus :
centroids=km.cluster_centers_

In [ ]:
predict = km.predict(df2)

In [ ]:
print(predict)

In [ ]:
from collections import Counter 
Counter(predict)

In [ ]:
SSE=km.inertia_
print(SSE)

In [ ]:
df2.head()

In [ ]:
# Using matplotlib.pyplot instead of seaborn.scatterplot to display the clusters.
plt.scatter(df2["Clean fuels and technologies for cooking acces (% pop)"], df2["Compulsory education duration (years)"], c=clusters)
plt.title('Data in Space', fontsize=14)
plt.xlabel("Clean fuels and technologies for cooking acces (% pop)",fontsize=14)
plt.ylabel("Compulsory education duration (years)",fontsize=14 )
plt.scatter(centroids[:, 0], centroids[:, 2], c='red',s=150, alpha=0.4)
plt.grid(True)

In [ ]:
# Visualisation du changement de nombre de cluster
km = KMeans(n_clusters=3, init='k-means++',n_init=10, random_state=50, max_iter=300,).fit(df2)

SSE=km.inertia_
print(SSE)

In [ ]:
#Courbe de la SSE en fonction du nombre de clusters
SSE_liste=[]
for i in range(2,10):
    km2=KMeans(n_clusters=i, init='k-means++',  n_init=1, random_state=50, max_iter=1000).fit(df2)
    SSE_liste.append(km2.inertia_)
    
#print(SSE_liste)

SSE_liste_random=[]
x=[]
for i in range(2,10):
    km3=KMeans(n_clusters=i, init='random',  n_init=1, random_state=50, max_iter=1000).fit(df2)
    SSE_liste_random.append(km3.inertia_)
    x.append(i)
    
#print(SSE_liste_random)

plt.plot(x, SSE_liste,label="k-means++")
plt.plot(x, SSE_liste_random,label="Random")
plt.xlabel("Nombre de clusters")
plt.ylabel("SSE")
plt.legend()
plt.show()

# Hierachical clustering :

In [ ]:
z = sch.linkage(df2, method = 'complete', metric='euclidean')

In [ ]:
z.shape

In [ ]:
plt.figure(figsize=(20, 40))
dendro = sch.dendrogram(z, 
                        orientation='right', 
                        leaf_rotation=0, 
                        leaf_font_size=14)
plt.title("Dendrogram")
plt.ylabel("Countries")
plt.xlabel("Distance")
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(z[:, 2], 'o-')
plt.grid(axis='y')
plt.show()